In [ ]:
# 전체 수정 완료본.
# 시간 표출 관련 코드 삭제
import cv2
import time
import numpy as np
import mediapipe as mp

# MediaPipe 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 랜드마크 인덱스 정의
LEFT_EYE_OUTER = 33
LEFT_IRIS_LEFT = 471
LEFT_IRIS_RIGHT = 469
LEFT_EYE_CENTER = 468
RIGHT_EYE_OUTER = 263
RIGHT_IRIS_LEFT = 476
RIGHT_IRIS_RIGHT = 474
RIGHT_EYE_CENTER = 473
LEFT_EAR = mp_pose.PoseLandmark.LEFT_EAR
RIGHT_EAR = mp_pose.PoseLandmark.RIGHT_EAR
LEFT_SHOULDER = mp_pose.PoseLandmark.LEFT_SHOULDER
RIGHT_SHOULDER = mp_pose.PoseLandmark.RIGHT_SHOULDER
LEFT_INDEX = mp_pose.PoseLandmark.LEFT_INDEX
RIGHT_ELBOW = mp_pose.PoseLandmark.RIGHT_ELBOW

# solvePnP용 3D 모델 포인트
def get_camera_matrix(w, h):
    focal_length = w
    center = (w / 2, h / 2)
    return np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype=np.float64)

model_points = np.array([
    (0.0, 0.0, 0.0),       # Nose tip
    (0.0, -63.6, -12.5),   # Chin
    (-43.3, 32.7, -26.0),  # Left eye corner
    (43.3, 32.7, -26.0),   # Right eye corner
    (-28.9, -28.9, -24.1), # Left mouth
    (28.9, -28.9, -24.1)   # Right mouth
], dtype=np.float32)

blink_threshold = 0.015
blink_cooldown = 5
blink_counter = 0
calibration_vals, is_calibrated = [], False
up_thresh, down_thresh = 0.0, 1.0

total_frames, center_frames = 0, 0
shoulder_warning_count, hand_warning_count = 0, 0
shoulder_prev_state = "CENTER"
hand_prev_state = "NONE"

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Webcam not opened.")
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w = frame.shape[:2]
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_result = face_mesh.process(rgb)
    pose_result = pose.process(rgb)

    gaze_dir, pitch_dir, turn_dir, shoulder_dir, hand_dir = "", "", "", "", ""
    feedbacks = []

    total_frames += 1

    if face_result.multi_face_landmarks:
        lm = face_result.multi_face_landmarks[0].landmark

        # 좌우 시선 분석
        left_eye_outer = np.array([lm[LEFT_EYE_OUTER].x * w, lm[LEFT_EYE_OUTER].y * h])
        left_iris_left = np.array([lm[LEFT_IRIS_LEFT].x * w, lm[LEFT_IRIS_LEFT].y * h])
        left_iris_right = np.array([lm[LEFT_IRIS_RIGHT].x * w, lm[LEFT_IRIS_RIGHT].y * h])
        left_ratio = np.linalg.norm(left_iris_left - left_eye_outer) / (np.linalg.norm(left_iris_right - left_iris_left) + 1e-6)

        right_eye_outer = np.array([lm[RIGHT_EYE_OUTER].x * w, lm[RIGHT_EYE_OUTER].y * h])
        right_iris_left = np.array([lm[RIGHT_IRIS_LEFT].x * w, lm[RIGHT_IRIS_LEFT].y * h])
        right_iris_right = np.array([lm[RIGHT_IRIS_RIGHT].x * w, lm[RIGHT_IRIS_RIGHT].y * h])
        right_ratio = np.linalg.norm(right_eye_outer - right_iris_right) / (np.linalg.norm(right_iris_right - right_iris_left) + 1e-6)

        gaze_h = "LEFT" if left_ratio < 0.45 else "RIGHT" if right_ratio < 0.48 else "CENTER"

        # 상하 시선 + 눈 감음 보정 포함
        eye_top, eye_bot = lm[159].y, lm[145].y
        opening = abs(eye_top - eye_bot)
        if opening < blink_threshold:
            blink_counter = blink_cooldown
            gaze_v = "CENTER"
        elif blink_counter > 0:
            blink_counter -= 1
            gaze_v = "CENTER"
        else:
            if not is_calibrated:
                calibration_vals.append(opening)
                if len(calibration_vals) == 30:
                    avg = np.mean(calibration_vals)
                    up_thresh, down_thresh = avg * 1.10, avg * 0.88
                    is_calibrated = True
                gaze_v = "CENTER"
            else:
                gaze_v = "UP" if opening > up_thresh else "DOWN" if opening < down_thresh else "CENTER"

        image_points = np.array([[lm[i].x * w, lm[i].y * h] for i in [1,152,33,263,78,308]], dtype=np.float32)
        cam_mtx = get_camera_matrix(w, h)
        success, rvec, tvec = cv2.solvePnP(model_points, image_points, cam_mtx, np.zeros((4,1)))
        pitch_dir = "CENTER"
        if success:
            rmat, _ = cv2.Rodrigues(rvec)
            pitch = np.degrees(np.arcsin(-rmat[2][1]))
            pitch_dir = "UP" if pitch < -14 else "DOWN" if pitch > 9 else "CENTER"

    if pose_result.pose_landmarks:
        plm = pose_result.pose_landmarks.landmark
        le, re = lm[LEFT_EYE_CENTER], lm[RIGHT_EYE_CENTER]
        le2e = abs(le.x - plm[LEFT_EAR].x)
        re2e = abs(re.x - plm[RIGHT_EAR].x)
        turn_dir = "LEFT" if le2e > re2e + 0.035 else "RIGHT" if re2e > le2e + 0.035 else "CENTER"

        diff = plm[LEFT_SHOULDER].y - plm[RIGHT_SHOULDER].y
        shoulder_dir = "LEFT UP" if diff > 0.03 else "RIGHT UP" if diff < -0.03 else "CENTER"

        if shoulder_dir != shoulder_prev_state and shoulder_prev_state == "CENTER":
            shoulder_warning_count += 1
        shoulder_prev_state = shoulder_dir

        hand_dir = "VISIBLE" if plm[LEFT_INDEX].visibility > 0.5 or plm[RIGHT_ELBOW].visibility > 0.5 else "NONE"
        if hand_dir != hand_prev_state and hand_prev_state == "NONE":
            hand_warning_count += 1
        hand_prev_state = hand_dir

        mp_drawing.draw_landmarks(frame, pose_result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    if turn_dir != "CENTER" or pitch_dir != "CENTER" or gaze_h != "CENTER" or gaze_v != "CENTER":
        feedbacks.append(("Look at the center.", (0, 0, 255)))
    else:
        feedbacks.append(("You're looking straight ahead.", (0, 255, 0)))
        center_frames += 1

    feedbacks.append(("Straighten your shoulders." if shoulder_dir != "CENTER" else "Shoulders are well aligned.", (0, 0, 255) if shoulder_dir != "CENTER" else (0, 255, 0)))
    feedbacks.append(("Lower your hands naturally." if hand_dir == "VISIBLE" else "Hands are properly placed.", (0, 0, 255) if hand_dir == "VISIBLE" else (0, 255, 0)))

    y = 30
    for text in [f"Gaze: {gaze_h} / {gaze_v}", f"Head: {turn_dir}", f"Pitch: {pitch_dir}", f"Shoulders: {shoulder_dir}", f"Hands: {hand_dir}"]:
        cv2.putText(frame, text, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)
        y += 25

    for fb, color in feedbacks:
        cv2.putText(frame, fb, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        y += 30

    gaze_score = int((center_frames / total_frames) * 100) if total_frames > 0 else 0
    shoulder_score = max(0, 50 - 5 * shoulder_warning_count)
    hand_score = max(0, 50 - 5 * hand_warning_count)
    total_score = gaze_score + shoulder_score + hand_score - 100

    score_texts = [
        f"Gaze Score: {gaze_score}/100",
        f"Shoulder Warnings: {shoulder_warning_count} (-{shoulder_warning_count*5})",
        f"Hand Warnings: {hand_warning_count} (-{hand_warning_count*5})",
        f"Total Score: {total_score}/100"
    ]
    for text in score_texts:
        y += 30
        cv2.putText(frame, text, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    cv2.imshow("Gaze Head Shoulder Tracking", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
